## Crypto Sentiment Analyzer

## Imports

In [1]:
# Imports
import os
import requests
import json
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [2]:
load_dotenv()

True

## Pull Data


#### Step 1: Import the OHLCV dataset into a Pandas DataFrame.

In [3]:
# Import the OHLCV dataset into a 

btc_url = 'https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD'
eth_url = 'https://api.alternative.me/v2/ticker/Ethereum/?convert=USD'
bnb_url = 'https://api.alternative.me/v2/ticker/BinanceCoin/?convert=USD'
xrp_url = 'https://api.alternative.me/v2/ticker/ripple/?convert=USD'
luna_url = 'https://api.alternative.me/v2/ticker/terra-luna/?convert=USD'
cardano_url ='https://api.alternative.me/v2/ticker/cardano/?convert=USD'
avax_url = 'https://api.alternative.me/v2/ticker/avalanche-2/?convert=USD'
dot_url = 'https://api.alternative.me/v2/ticker/polkadot/?convert=USD'
doge_url = 'https://api.alternative.me/v2/ticker/doge/?convert=USD'
shiba_url= 'https://api.alternative.me/v2/ticker/shiba-inu/?convert=USD'
api_key = os.getenv('API_Key')

In [4]:
btc_response = requests.get(btc_url).json()
eth_response= requests.get(eth_url).json()
bnb_response= requests.get(bnb_url).json()
xrp_response= requests.get(xrp_url).json()
luna_response = requests.get(luna_url).json()
cardano_response = requests.get(cardano_url).json()
avax_response = requests.get(avax_url).json()
dot_response= requests.get(dot_url).json()
doge_response = requests.get(doge_url).json()
shiba_response = requests.get(shiba_url).json()



In [5]:
alpaca_api_key =  'API_KEY'
alpaca_private_key = 'API_PRIVATE_KEY'


alpaca= tradeapi.REST(
alpaca_api_key,
alpaca_private_key,
api_version= "v2")

## Pull BTC data and build BTC.df

In [8]:

# Filter the date index and close columns
tickers = ['BTCUSD']
# Use the pct_change function to generate returns from close prices
timeframe = '1Day'

# Drop all NaN values from the DataFrame

start_date = pd.Timestamp('2018-01-01', tz= "America/New_York").isoformat()
end_date = pd.Timestamp('2022-06-19', tz="America/New_York").isoformat()


In [10]:
end_date

'2022-06-19T00:00:00-04:00'

In [11]:
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
BTC_df = alpaca.get_crypto_bars(
tickers,
# timeframe,
timeframe, 
# start=start_date,
# end=end_date,
start=start_date, 
end=end_date,
exchanges = 'CBSE').df

BTC_df

HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v1beta1/crypto/bars?timeframe=1Day&start=2018-01-01T00%3A00%3A00-05%3A00&end=2022-06-19T00%3A00%3A00-04%3A00&exchanges=CBSE&symbols=BTCUSD

In [9]:
# Drop all NaN values from the DataFrame
BTC_df = BTC_df.dropna()

# Review the DataFrame
BTC_df.head()


,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2018-01-01 06:00:00+00:00,CBSE,13480.00,13999.00,13005.00,13194.02,6200.047084,49282,13515.969795,BTCUSD
2018-01-02 06:00:00+00:00,CBSE,14781.52,15198.88,14761.00,15098.88,4422.372928,36177,15002.191483,BTCUSD
2018-01-03 06:00:00+00:00,CBSE,15098.87,15400.00,14628.00,15098.14,12001.450099,91543,14992.624035,BTCUSD
2018-01-04 06:00:00+00:00,CBSE,15145.00,15176.86,14819.78,15038.00,3086.394104,26818,15022.817179,BTCUSD
2018-01-05 06:00:00+00:00,CBSE,16960.01,16999.00,16410.75,16594.08,4507.400878,43035,16696.958061,BTCUSD


In [10]:
BTC_df = BTC_df.loc[:,["close"]]

BTC_df.head()

,close
timestamp,
2018-01-01 06:00:00+00:00,13194.02
2018-01-02 06:00:00+00:00,15098.88
2018-01-03 06:00:00+00:00,15098.14
2018-01-04 06:00:00+00:00,15038.00
2018-01-05 06:00:00+00:00,16594.08


## CREATE SIGNAL

In [11]:
# Calculate the daily returns using the closing prices and the pct_change function
BTC_df["Daily Return"] = BTC_df["close"].pct_change()


# Display sample data
BTC_df.head()

,close,Daily Return
timestamp,,
2018-01-01 06:00:00+00:00,13194.02,NaN
2018-01-02 06:00:00+00:00,15098.88,0.144373
2018-01-03 06:00:00+00:00,15098.14,-0.000049
2018-01-04 06:00:00+00:00,15038.00,-0.003983
2018-01-05 06:00:00+00:00,16594.08,0.103477


In [12]:
BTC_df = BTC_df.dropna()

BTC_df.head()

,close,Daily Return
timestamp,,
2018-01-02 06:00:00+00:00,15098.88,0.144373
2018-01-03 06:00:00+00:00,15098.14,-0.000049
2018-01-04 06:00:00+00:00,15038.00,-0.003983
2018-01-05 06:00:00+00:00,16594.08,0.103477
2018-01-06 06:00:00+00:00,17098.99,0.030427


## Create Signal Column

In [13]:
# Initialize investment signal tally

BTC_signal = 0

# Initialize trade_type column for buys and sell
BTC_df["BTC_score"] = np.nan

# Initialize variable to hold the previous_price
previous_BTC_price = 0

# Loop through the Pandas DataFrame and initiate a score for each iteration


for index, row in BTC_df.iterrows():
    
    # add 1 if the current day price is greater than the previous day
        
    if row["close"] < previous_BTC_price:
        BTC_signal = BTC_signal + 1
        BTC_df.loc[index, "BTC_score"] = 1    
          
    # subtract 1 if the current day price is greater than the previous day
    if row["close"] > previous_BTC_price:
        BTC_signal = BTC_signal - 1
        BTC_df.loc[index, "BTC_score"] = 0    
    
    # set the previous_price variable to the close of current row
    previous_BTC_price = row["close"]

    # if the index is the last index of the Data
    if index == BTC_df.index[-1]:
        BTC_df.loc[index, "BTC_score"] = 1

In [14]:
BTC_df.head()

,close,Daily Return,BTC_score
timestamp,,,
2018-01-02 06:00:00+00:00,15098.88,0.144373,0.0
2018-01-03 06:00:00+00:00,15098.14,-0.000049,1.0
2018-01-04 06:00:00+00:00,15038.00,-0.003983,1.0
2018-01-05 06:00:00+00:00,16594.08,0.103477,0.0
2018-01-06 06:00:00+00:00,17098.99,0.030427,0.0


## Create Tradetype column

In [15]:
# Initialize trade_type column for buys and sell
BTC_df["trade_type"]= np.nan


# Loop through the Pandas DataFrame and initiate
for index, row in BTC_df.iterrows():
    
    # buy if the previous price is 0
    if previous_BTC_price == 0:
        BTC_df.loc[index, "trade_type"] = 1

    # buy if the current day price is less than previous close
    elif row["close"] < previous_BTC_price:
        BTC_df.loc[index, "trade_type"] = 1

    # hold if the current day price is greater than previous close
    elif row["close"] > previous_BTC_price:
        BTC_df.loc[index, "trade_type"] = 0

    # as well, hold if the current day price is equal to previous close
    else:
        BTC_df.loc[index, "trade_type"] = 0
    
    # set the previous_price variable to the close of current row
    BTC_price = row["close"]

    # if the index is the last index of the Data
    if index == BTC_df.index[-1]:
        BTC_df.loc[index, "trade_type"] = 1

In [16]:
BTC_df

,close,Daily Return,BTC_score,trade_type
timestamp,,,,
2018-01-02 06:00:00+00:00,15098.88,0.144373,0.0,1.0
2018-01-03 06:00:00+00:00,15098.14,-0.000049,1.0,1.0
2018-01-04 06:00:00+00:00,15038.00,-0.003983,1.0,1.0
2018-01-05 06:00:00+00:00,16594.08,0.103477,0.0,1.0
2018-01-06 06:00:00+00:00,17098.99,0.030427,0.0,1.0
...,...,...,...,...
2022-03-10 06:00:00+00:00,38783.79,-0.011152,1.0,1.0
2022-03-11 06:00:00+00:00,39178.11,0.010167,0.0,0.0
2022-03-12 06:00:00+00:00,39128.83,-0.001258,1.0,0.0


## Pull ETH Data and build ETH.DF

In [17]:

# Filter the date index and close columns
tickers = ['ETHUSD']
# Use the pct_change function to generate returns from close prices
timeframe = '1Day'

# Drop all NaN values from the DataFrame

start_date = pd.Timestamp('2018-01-01', tz= "America/New_York").isoformat()
end_date = pd.Timestamp('2022-03-15', tz="America/New_York").isoformat()

In [18]:
start_date

'2018-01-01T00:00:00-05:00'

In [19]:
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
ETH_df = alpaca.get_crypto_bars(
tickers,
# timeframe,
timeframe, 
start = start_date,
end = end_date,
exchanges = 'CBSE').df    

ETH_df.head(10)

,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2018-01-01 06:00:00+00:00,CBSE,759.03,878.50,759.02,842.00,160273.229811,65625,831.412135,ETHUSD
2018-01-02 06:00:00+00:00,CBSE,841.99,874.90,820.40,865.00,161504.891191,96283,855.606217,ETHUSD
2018-01-03 06:00:00+00:00,CBSE,938.16,940.00,920.10,936.01,64663.450454,31322,932.278314,ETHUSD
2018-01-04 06:00:00+00:00,CBSE,949.00,996.90,942.00,994.00,67036.333366,38214,979.835753,ETHUSD
2018-01-05 06:00:00+00:00,CBSE,994.00,1015.00,935.00,969.27,188326.705380,110141,974.989335,ETHUSD
2018-01-06 06:00:00+00:00,CBSE,1006.68,1075.00,1003.02,1075.00,57881.009164,27806,1032.296385,ETHUSD
2018-01-07 06:00:00+00:00,CBSE,1118.30,1228.99,1110.41,1192.80,110954.102938,53816,1176.104340,ETHUSD
2018-01-08 06:00:00+00:00,CBSE,1192.80,1225.50,950.00,1137.01,233382.887766,119888,1111.666808,ETHUSD
2018-01-09 06:00:00+00:00,CBSE,1290.01,1386.99,1244.01,1356.69,119254.987203,57975,1321.254101,ETHUSD


In [20]:
ETH_df = ETH_df.loc[:,["close"]]
ETH_df

,close
timestamp,
2018-01-01 06:00:00+00:00,842.00
2018-01-02 06:00:00+00:00,865.00
2018-01-03 06:00:00+00:00,936.01
2018-01-04 06:00:00+00:00,994.00
2018-01-05 06:00:00+00:00,969.27
...,...
2022-03-10 06:00:00+00:00,2566.29
2022-03-11 06:00:00+00:00,2591.04
2022-03-12 06:00:00+00:00,2584.63


In [21]:
# Calculate the daily returns using the closing prices and the pct_change function
ETH_df["Daily Return"] = ETH_df["close"].pct_change()


# Display sample data
ETH_df.head()

,close,Daily Return
timestamp,,
2018-01-01 06:00:00+00:00,842.00,NaN
2018-01-02 06:00:00+00:00,865.00,0.027316
2018-01-03 06:00:00+00:00,936.01,0.082092
2018-01-04 06:00:00+00:00,994.00,0.061954
2018-01-05 06:00:00+00:00,969.27,-0.024879


In [22]:
# Drop all NaN values from the DataFrame
ETH_df = ETH_df.dropna()

# Review the DataFrame
ETH_df.head()


,close,Daily Return
timestamp,,
2018-01-02 06:00:00+00:00,865.00,0.027316
2018-01-03 06:00:00+00:00,936.01,0.082092
2018-01-04 06:00:00+00:00,994.00,0.061954
2018-01-05 06:00:00+00:00,969.27,-0.024879
2018-01-06 06:00:00+00:00,1075.00,0.109082


In [23]:
# Initialize investment signal tally

ETH_signal = 0

# Initialize trade_type column for buys and sell
ETH_df["ETH_score"] = np.nan

# Initialize variable to hold the previous_price
previous_ETH_price = 0

# Loop through the Pandas DataFrame and initiate a score for each iteration


for index, row in ETH_df.iterrows():
    
    # add 1 if the current day price is greater than the previous day
        
    if row["close"] < previous_ETH_price:
        ETH_signal = ETH_signal + 1
        ETH_df.loc[index, "ETH_score"] = 1    
          
    # subtract 1 if the current day price is greater than the previous day
    if row["close"] > previous_ETH_price:
        ETH_signal = ETH_signal - 1
        ETH_df.loc[index, "ETH_score"] = 0    
    
    # set the previous_price variable to the close of current row
    previous_ETH_price = row["close"]

    # if the index is the last index of the Data
    if index == ETH_df.index[-1]:
        ETH_df.loc[index, "ETH_score"] = 1

/Users/derekhall/opt/anaconda3/envs/dev/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/derekhall/opt/anaconda3/envs/dev/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/derekhall/opt/anaconda3/envs/dev/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

In [24]:
len(ETH_df)

1533

In [25]:
len(BTC_df)

1533

## Pull BNB Data and build BNB.DF

In [26]:

# Filter the date index and close columns
#tickers = ['BNB']
# Use the pct_change function to generate returns from close prices
#timeframe = '1D'

#limit_rows= 1000
# Drop all NaN values from the DataFrame

#start_date = pd.Timestamp('2018-01-01', tz= "America/New_York").isoformat()
#end_date = pd.Timestamp('2022-03-15', tz="America/New_York").isoformat()

In [27]:
#start_date

In [28]:
#from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
#BNB_df = alpaca.get_bars(
#tickers,
# timeframe,
#TimeFrame(1, TimeFrameUnit.Day), 
# start=start_date,
# end=end_date,
#start='2018-01-01', 
#end='2022-03-15',
#limit=limit_rows).df

#BNB_df.head()

In [29]:
# Drop all NaN values from the DataFrame
#BNB_df = BNB_df.dropna()

# Review the DataFrame
#BNB_df.head()

## Pull DOGE data and build DOGE.DF

## CREATE FINAL DF

In [34]:
# pd.concat([df_1, df_2], axis=1)
new_ETH_df=ETH_df
new_ETH_df.index
new_BTC_df=BTC_df
new_BTC_df.index

Learning_df = pd.concat([new_ETH_df.loc[:,["ETH_score"]], new_BTC_df.loc[:,["BTC_score"]]],axis=1)

In [35]:
Learning_df

,ETH_score,BTC_score
timestamp,,
2018-01-02 06:00:00+00:00,0.0,0.0
2018-01-03 06:00:00+00:00,0.0,1.0
2018-01-04 06:00:00+00:00,0.0,1.0
2018-01-05 06:00:00+00:00,1.0,0.0
2018-01-06 06:00:00+00:00,0.0,0.0
...,...,...
2022-03-10 06:00:00+00:00,1.0,1.0
2022-03-11 06:00:00+00:00,0.0,0.0
2022-03-12 06:00:00+00:00,1.0,1.0


In [37]:
Learning_df['trade_type'] = new_BTC_df.loc[:,['trade_type']]

In [38]:
Learning_df

,ETH_score,BTC_score,trade_type
timestamp,,,
2018-01-02 06:00:00+00:00,0.0,0.0,1.0
2018-01-03 06:00:00+00:00,0.0,1.0,1.0
2018-01-04 06:00:00+00:00,0.0,1.0,1.0
2018-01-05 06:00:00+00:00,1.0,0.0,1.0
2018-01-06 06:00:00+00:00,0.0,0.0,1.0
...,...,...,...
2022-03-10 06:00:00+00:00,1.0,1.0,1.0
2022-03-11 06:00:00+00:00,0.0,0.0,0.0
2022-03-12 06:00:00+00:00,1.0,1.0,0.0


## Implement SKLEARN

In [42]:
# Import module
from sklearn.model_selection import train_test_split

# Split training and testing sets
# Create X, or features DataFrame
features = Learning_df[['ETH_score','BTC_score']]

# Create y, or target DataFrame
target = Learning_df['trade_type']

# Use train_test_split to separate the data
training_features, testing_features, training_targets, testing_targets = train_test_split(features, target)

In [43]:
training_targets.head(50)

timestamp
2021-07-18 05:00:00+00:00    1.0
2019-12-13 06:00:00+00:00    1.0
2022-01-15 06:00:00+00:00    0.0
2021-08-10 05:00:00+00:00    0.0
2020-05-01 05:00:00+00:00    1.0
2018-11-14 06:00:00+00:00    1.0
2021-03-05 06:00:00+00:00    0.0
2020-10-03 05:00:00+00:00    1.0
2020-11-07 06:00:00+00:00    1.0
2020-04-30 05:00:00+00:00    1.0
2019-03-28 05:00:00+00:00    1.0
2018-11-23 06:00:00+00:00    1.0
2019-05-17 05:00:00+00:00    1.0
2018-07-03 05:00:00+00:00    1.0
2022-02-20 06:00:00+00:00    0.0
2021-01-16 06:00:00+00:00    1.0
2019-01-17 06:00:00+00:00    1.0
2020-07-28 05:00:00+00:00    1.0
2019-02-12 06:00:00+00:00    1.0
2021-12-22 06:00:00+00:00    0.0
2019-01-12 06:00:00+00:00    1.0
2020-08-01 05:00:00+00:00    1.0
2021-10-22 05:00:00+00:00    0.0
2020-01-10 06:00:00+00:00    1.0
2018-09-05 05:00:00+00:00    1.0
2018-11-02 05:00:00+00:00    1.0
2021-12-26 06:00:00+00:00    0.0
2018-05-03 05:00:00+00:00    1.0
2018-03-13 05:00:00+00:00    1.0
2018-02-19 06:00:00+00:00    1.0


In [44]:
training_features

,ETH_score,BTC_score
timestamp,,
2021-07-18 05:00:00+00:00,1.0,1.0
2019-12-13 06:00:00+00:00,1.0,0.0
2022-01-15 06:00:00+00:00,1.0,1.0
2021-08-10 05:00:00+00:00,0.0,1.0
2020-05-01 05:00:00+00:00,0.0,0.0
...,...,...
2019-10-07 05:00:00+00:00,0.0,0.0
2020-01-22 06:00:00+00:00,1.0,1.0
2018-03-31 05:00:00+00:00,1.0,1.0


In [45]:
logistic_regression_model = LogisticRegression()

In [46]:
# Fit the model
logistic_regression_model.fit(training_features, training_targets)

LogisticRegression()

In [47]:
# Fit the model
logistic_regression_model.fit(training_features, training_targets)

LogisticRegression()

In [48]:
# Generate predictions from the model we just fit
predictions = logistic_regression_model.predict(training_features)

# Convert those predictions (and actual values) to a DataFrame
results_df = pd.DataFrame({"Prediction": predictions, "Actual": training_targets})
results_df

,Prediction,Actual
timestamp,,
2021-07-18 05:00:00+00:00,1.0,1.0
2019-12-13 06:00:00+00:00,1.0,1.0
2022-01-15 06:00:00+00:00,1.0,0.0
2021-08-10 05:00:00+00:00,1.0,0.0
2020-05-01 05:00:00+00:00,1.0,1.0
...,...,...
2019-10-07 05:00:00+00:00,1.0,1.0
2020-01-22 06:00:00+00:00,1.0,1.0
2018-03-31 05:00:00+00:00,1.0,1.0


In [49]:
# Import the accuracy_score function
from sklearn.metrics import accuracy_score

# Calculate the model's accuracy on the test dataset
accuracy_score(training_targets, predictions)

0.7893820713664056